In [3]:
import pandas as pd
df = pd.read_csv("mooc_dataset/coursera_reviews.csv")


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1454711 entries, 0 to 1454710
Data columns (total 5 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   Unnamed: 0    1454711 non-null  int64 
 1   reviewers     1454711 non-null  object
 2   date_reviews  1454711 non-null  object
 3   rating        1454711 non-null  int64 
 4   course_id     1454711 non-null  object
dtypes: int64(2), object(3)
memory usage: 55.5+ MB


In [7]:
df.nunique()

Unnamed: 0      1454711
reviewers        287808
date_reviews       1892
rating                5
course_id           604
dtype: int64

In [ ]:
# Drop unnecessary columns
dataset_cleaned = df.drop(columns=['Unnamed: 0', 'date_reviews', 'rating'])

# Remove duplicate rows
dataset_cleaned = dataset_cleaned.drop_duplicates()

# Display the cleaned dataset
dataset_cleaned.head()
d

,reviewers,course_id
0,By Robert S,google-cbrs-cpi-training
1,By Gabriel E R,google-cbrs-cpi-training
2,By Jacob D,google-cbrs-cpi-training
3,By Dale B,google-cbrs-cpi-training
4,By Sean G,google-cbrs-cpi-training


In [18]:
dataset_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 506116 entries, 0 to 1454644
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   reviewers  506116 non-null  object
 1   course_id  506116 non-null  object
dtypes: object(2)
memory usage: 11.6+ MB


In [17]:
dataset_cleaned.nunique()

reviewers    287808
course_id       604
dtype: int64

In [9]:
# Group courses by reviewers to create a transactional dataset
transactional_data = dataset_cleaned.groupby('reviewers')['course_id'].apply(list).reset_index()

# Rename columns for clarity
transactional_data.columns = ['reviewer', 'courses']

# Display the transactional dataset
transactional_data.head()

,reviewer,courses
0,By \t M N H,[grammar-punctuation]
1,By \t M R I,[python-data]
2,By \t M S R,[guitar]
3,By \t N A F B,[uva-darden-digital-product-management]
4,By \t N H P,[speak-english-professionally]


In [ ]:
from mlxtend.preprocessing import TransactionEncoder

# Extract the list of courses for each reviewer
courses_list = transactional_data['courses'].tolist()

# Use TransactionEncoder to encode the data
te = TransactionEncoder()
te_data = te.fit_transform(courses_list)
df_encoded = pd.DataFrame(te_data, columns=te.columns_)

# Display the one-hot encoded dataset
df_encoded.head()

,aboriginal-education,access-control-sscp,accounting-analytics,accounting-data-analytics-python,actualizacion-manejo-diabetes-tipo-2,addiction-and-the-brain,addiction-treatment,advanced-valuation-and-strategy,agile-atlassian-jira,ai-for-everyone,...,what-is-compliance,what-is-datascience,what-is-social,wind-energy,wine,womens-health-human-rights,write-a-feature-length-screenplay-for-film-or-television,write-your-first-novel,writing-editing-words,writing-for-business
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [14]:
from mlxtend.frequent_patterns import apriori, association_rules
encoded_data = df_encoded
# Find frequent itemsets using Apriori
frequent_itemsets = apriori(encoded_data, min_support=0.01, use_colnames=True)

# Generate association rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0, num_itemsets = len(frequent_itemsets))

rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(python-data),(python),0.037254,0.048862,0.010205,0.273923,5.605999,1.0,0.008384,1.309967,0.853413,0.134429,0.236622,0.241384
1,(python),(python-data),0.048862,0.037254,0.010205,0.208846,5.605999,1.0,0.008384,1.216888,0.863828,0.134429,0.178232,0.241384
